In [ ]:
from library_dicom.rtss_processor.model.RTSS_Reader import RTSS_Reader
from library_dicom.rtss_processor.model.Mask_To_RTSS import Mask_To_RTSS
from library_dicom.dicom_processor.model.Series import Series
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
CT_path=r'C:\Users\wendy\Documents\STAGE\DicomRT\201802409 BARAT GALPIN ELIANE\103893708 Tepscan FDG\CT CT STD 2.5mm'

In [ ]:
serie_CT = Series(CT_path)
#serie_CT.get_series_details()

### Get data/ UID to build new RTSS from a CT serie 

In [ ]:
study_instance_uid = serie_CT.get_series_details()['study']['StudyInstanceUID']
serie_instance_uid = serie_CT.get_series_details()['series']['SeriesInstanceUID']
print(study_instance_uid)
print(serie_instance_uid)

In [ ]:
frame_of_reference_uid = serie_CT.get_FrameOfReferenceIUD()
print(frame_of_reference_uid)

In [ ]:
referenced_sop_class_uid = serie_CT.get_ReferencedSOPClassUID()
print(referenced_sop_class_uid)

In [ ]:
list_all_uid = serie_CT.get_all_SOPInstanceIUD()


In [ ]:
pixel_spacing = [1.171875, 1.171875, -2.5]
image_position = [-300.0, -300.0, 325.0]

### RTSTRUCT file -> Mask 

In [ ]:
rtss_path = r'C:\Users\wendy\Documents\STAGE\DicomRT\201802409 BARAT GALPIN ELIANE\103893708 Tepscan FDG\RTSTRUCT PETVCAR RTSS RTSS'

In [ ]:
rtss = RTSS_Reader(rtss_path)

In [ ]:
rtss.get_number_of_roi()

In [ ]:
list_SOPInstance = rtss.get_list_all_SOP_Instance_UID()

In [ ]:
from time import process_time
t1_start = process_time()
mask_4D = rtss.rtss_to_4D_mask([512,512,415], pixel_spacing, image_position, list_SOPInstance,  matrix_4D=True)
t1_stop = process_time() 
print("diff début fin :", t1_stop-t1_start)  

In [ ]:
plt.imshow(mask_4D[:,:,308,0]) #0 pour ROI 1
plt.show()
#3Dslicer <= 309 slice |  308 ième => Python 

### Mask -> RTSTRUCT file 


In [ ]:
from library_dicom.rtss_processor.model.RTSS import RTSS
import pydicom

In [ ]:
new_rtss = RTSS('mask', 'filename', CT_path)

#### SET 

In [ ]:
new_rtss.set_ReferencedFrameOfReferenceSequence(frame_of_reference_uid, study_instance_uid,  serie_instance_uid, referenced_sop_class_uid, list_all_uid)

#### set StructureSetROISequence, set for each ROI 

In [ ]:
for number_roi in range(1,11):
    
    new_rtss.set_StructureSetROISequence(number_roi, frame_of_reference_uid, 'ROIName', 'ROIDescription', 100000, 'SEMI-AUTOMATIC')

#### set ROIContourSequence, loop on each ROI


In [ ]:
list_all_uid = serie_CT.get_all_SOPInstanceIUD()

number_of_roi = mask_4D.shape[3]
mask_to_roi = Mask_To_RTSS(mask_4D)

for number_roi in range(1, number_of_roi +1):
    contours, referenced_instance_UID = mask_to_roi.pixel_to_spatial(number_roi, [-300.0, -300.0, 325.0], [1.171875, 1.171875, -2.5], list_all_uid)
    new_rtss.set_ROIContourSequence(number_roi, [0,0,255], referenced_sop_class_uid,  referenced_instance_UID, 'CLOSED_PLANAR',  contours)
    

#### set RTROIObservationsSequence, set for each ROI 

In [ ]:
for number_roi in range(1,11):
    

    new_rtss.set_RTROIObservationSequence(number_roi, number_roi, 'ROIObservationLabel', 'RTROIInterpretedType')

### Save New RTSTRUCT

In [ ]:
import os 
new_rtss_directory = r'C:\Users\wendy\Documents\STAGE\DicomRT\201802409 BARAT GALPIN ELIANE\103893708 Tepscan FDG\new_rtss'
filename = r'RT1.dcm'


In [ ]:
os.path.join(new_rtss_directory, filename)

In [ ]:
new_rtss.save_as(os.path.join(new_rtss_directory, filename))

In [ ]:
dicom_rt =  pydicom.dcmread(os.path.join(new_rtss_directory, filename))

In [ ]:
dicom_rt.ROIContourSequence[1]